# 509 Final Project

This notebook queries the URLs from the MySQL table, scrapes the HTML article contents, then inserts it into a pandas dataframe. The scraped data is then persisted in two ways: 1) The HTML content is written it back to another column in the MySQL table; 2) A copy of the full DF is written to a CSV file for further processing (i.e., preprocessing).

## Globally import libraries

Libraries needed mostly pertain to MySQL database connection and dataframe manipulation.

In [1]:
import numpy as np
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt
import os
import shutil
import re
import logging
import time
import zipfile
import requests
from bs4 import BeautifulSoup
import datetime as dt
import re
import regex as rex
from collections import defaultdict, Counter
import random
import json

# Set pandas global options
pd.options.display.max_rows = 17

## Initiate MySQL connection

Open connection to existing MySQL schema and table. Pull masked user and password from environment variable.

In [2]:
'''Set local environment variables to hide user name & password citation:
https://www.geeksforgeeks.org/how-to-hide-sensitive-credentials-using-python/
'''
user_name = os.environ['MySQLUSRAC']
user_pass = os.environ['MySQLPWDAC']

# Instantiate connection
db_conn = mysql.connect(host='localhost',
                        port=int(3306),
                        user=user_name,
                        passwd=user_pass,
                        db='599_capstone')

# Create a cursor object
cursor = db_conn.cursor()

Display schema tables to ensure the connection was successful.

In [3]:
tbl_names = pd.read_sql('SHOW TABLES', db_conn)

display(tbl_names)
print(type(tbl_names))

,Tables_in_599_capstone
0,nar_temp
1,news_articles


<class 'pandas.core.frame.DataFrame'>


Use pandas to select all articles from the MySQL database

In [4]:
slct_tbl_full_df02 = pd.read_sql(
                                '''
                                SELECT * FROM news_articles
                                ''',
                                db_conn)

Establish working directories for saving dataframes as CSV files

In [5]:
'''Dir nav citation:
https://softhints.com/python-change-directory-parent/
'''
curr_dir = os.path.abspath(os.curdir)
print(curr_dir)
os.chdir("..")
up1_dir = os.path.abspath(os.curdir)
print(up1_dir)

C:\Users\acarr\Documents\GitHub\599_team_project\deliverables
C:\Users\acarr\Documents\GitHub\599_team_project


Get current date/time to append to file name string

In [6]:
today = dt.datetime.today()
today= str(today)
today = today.replace(':', '-').replace('.', '').replace(' ', '_')
print(today)
print(type(today))

2023-07-08_15-56-01966455
<class 'str'>


Establish full file name path

In [7]:
data_location = 'data'

scraped_file_name = f'data_scraped_amc_{today}.csv'
all_file_name = f'data_all_amc_{today}.csv'

scraped_file_path01 = os.path.join(up1_dir, data_location, scraped_file_name)
all_file_path01 = os.path.join(up1_dir, data_location, all_file_name)

print(f'CSV file path: {scraped_file_path01}')
print(f'CSV file path: {all_file_path01}')

CSV file path: C:\Users\acarr\Documents\GitHub\599_team_project\data\data_scraped_amc_2023-07-08_15-56-01966455.csv
CSV file path: C:\Users\acarr\Documents\GitHub\599_team_project\data\data_all_amc_2023-07-08_15-56-01966455.csv


Send dataframe to CSV

In [8]:
slct_tbl_full_df02.to_csv(all_file_path01, index=False)

### Establish logging policy

Configure logging so that connection errors can be reviewed. Logging output is sent to the pymysql.log file.

In [9]:
'''Logging citations (see additional code in following code blocks:
OpenAI. (2021). ChatGPT [Computer software]. https://openai.com/;
https://docs.python.org/3/howto/logging.html#logging-basic-example;
https://docs.python.org/3/howto/logging.html#logging-to-a-file;
https://docs.python.org/3/howto
/logging-cookbook.html#using-a-rotating-log-file-handler;
https://docs.python.org/3/howto
/logging-cookbook.html#using-a-timed-rotating-file-handler
'''
# Set up logging
logging.basicConfig(level=logging.INFO,
                    filename='pymysql.log',
                    filemode='a',
                    format='''>>>>>>>>>>>>>><<<<<<<<<<<<<<\n%(asctime)s - %(levelname)s - %(message)s''')

### Read URLs from MySQL table to perform web scraping

In [10]:
nat_tbl_name = 'nar_temp'
nwa_tbl_name = 'news_articles'

Run the custom function through a `for` loop to load the URLs in batches from MySQL, scrape the article content from the website, then persist the scraped data back to MySQL. The script first tries to ID whether a JSON object is available for simplied parsing; if there isn't one, the script parses the HTML using BeautifulSoup, iteratively searching for different tags based on the structure of each source website. If all parsing fails, then None is written to `article_text` and an error string is displayed; otherwise the scraped text is written to the feature.

In [11]:
'''Connect to MqSQL table in batches citation:
OpenAI. (2021). ChatGPT [Computer software]. https://openai.com/
'''

# Batch size (number of URLs to process at a time)
batch_size = 15000

# Get the total number of URLs in the table
count_query = f'''
SELECT COUNT(*) FROM {nwa_tbl_name}
WHERE article_text IS NULL'''

cursor.execute(count_query)
total_urls = cursor.fetchone()[0]
print(f'URL Count: {total_urls}')

# Start timer
start_time = dt.datetime.today()

# Calculate the number of batches required
num_batches = (total_urls // batch_size) + 1

# Process URLs in batches
for batch in range(num_batches):
    offset = batch * batch_size

    # Retrieve URLs from the MySQL table in the current batch
    query = f'''
    SELECT url FROM {nwa_tbl_name}
    WHERE article_text IS NULL
    AND LEFT(url, 19) <> 'https://disneyparks'
    LIMIT {batch_size}
    OFFSET {offset}
    '''
    
    cursor.execute(query)
    urls = cursor.fetchall()
    print(f'URL batch size: {len(urls)}')
    print(query)

    # Iterate over the URLs and scrape their contents
    for idx, url in enumerate(urls):
        url = url[0]  # Extract the URL from the tuple

        # Make an HTTP request to the URL
        response = requests.get(url)
        print(f'URL #{idx} Response: {response.status_code}; URL: {url} ', end='')
        time.sleep(5 + 10 * random.random())

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
            
            '''_______________________________________________'''

            # Extract the raw text from the HTML
            #print(soup.prettify())

            # Check for available JSON object
            try:
                script_tag = soup.find('script', {'type': 'application/ld+json'})
                if script_tag != None:
                    article_json = json.loads(script_tag.string)
                    article_content = article_json['articleBody']
                    #print('JSON article content:', article_content)

            # If no JSON object available, use Beautiful Soup to look for available 
            # HTML tags
            except:
                json_err02 = '; Missing JSON object'
                article_body = soup.find('div', class_='article__content-container')

                if article_body is None: # Fox & breitbert(sometimes)
                    article_body = soup.find('p', class_="speakable")
                    if article_body is None: # breitbert(most)
                        article_body = soup.find('div', class_='entry-content')
                        if article_body is None: # WashPost
                            article_body = soup.find('div',class_='article-body')
                            if article_body is None: # USA Today
                                article_body = soup.find('div',class_='gnt_ar_b')
                                if article_body is None: # USA Today
                                    article_body = soup.find('div',id='primary')
                                    if article_body is None: # ABC
                                        article_body = soup.find('div',class_='FITT_Article_main__body')
                                        if article_body is None: # BuzzFeed
                                            article_body = soup.find('article')
                                            if article_body is None: # BuzzFeed
                                                article_body = soup.find('div',id='1-main-content')
                                                if article_body is None: # CNBC
                                                    article_body = soup.find('div',class_='ArticleBody-articleBody')

                if article_body is not None:
                    article_content = article_body.get_text()
                    #print('Article body content:', article_content)
                    #print('Rejoice, parse was successful!')
                else:
                    try:
                        print(json_err02)
                    except:
                        pass
                    print(f'^^^Parse not successful^^^')
                    article_content = None
                    #print('Article not parse:', article_content)

            '''_______________________________________________'''

            # Update the MySQL table with the scraped text
            update_query = '''
            UPDATE news_articles SET article_text = %s
            WHERE url = %s
            '''
            
            #print('.', end='')
            #print(update_query)
            #print('Article content to post:', article_content, '\n')
            cursor.execute(update_query, (article_content, url))
            db_conn.commit()
            
# End timer script
end_time = dt.datetime.today()
time_elapse = end_time - start_time
print(f'Start Time = {start_time}')
print(f'End Time = {end_time}')
print(f'Elapsed Time = {time_elapse}')

URL Count: 35928
URL batch size: 5000

    SELECT url FROM news_articles
    WHERE article_text IS NULL
    AND LEFT(url, 19) <> 'https://disneyparks'
    LIMIT 5000
    OFFSET 0
    
URL #0 Response: 200; URL: https://abcnews.go.com/GMA/Living/video/97-year-cycled-100000-miles-100387792 ; Missing JSON object
^^^Parse not successful^^^
URL #1 Response: 403; URL: https://www.wsj.com/livecoverage/stock-market-today-dow-jones-06-26-2023 URL #2 Response: 403; URL: https://www.wsj.com/livecoverage/stock-market-today-dow-jones-06-27-2023 URL #3 Response: 200; URL: https://reviewed.usatoday.com/gaming/content/nzxt-relay-review-comfortable-fit-and-clear-comms-but-too-expensive ; Missing JSON object
^^^Parse not successful^^^
URL #4 Response: 200; URL: https://abcnews.go.com/Travel/video/ride-virgin-galactics-1st-paying-flight-100536953 ; Missing JSON object
^^^Parse not successful^^^
URL #5 Response: 200; URL: https://abcnews.go.com/GMA/Living/video/bagel-shop-asked-customers-silly-dance-enter

KeyboardInterrupt: 

### Send MySQL records to CSV

Use pandas to select all articles from the MySQL database where `article_text` is not missing

In [12]:
slct_tbl_full_df01 = pd.read_sql(
                                '''
                                SELECT * FROM news_articles
                                WHERE article_text IS NOT NULL
                                ''',
                                db_conn)

Send dataframe to CSV

In [13]:
slct_tbl_full_df01.to_csv(scraped_file_path01, index=False)

Display dataframe attributes and header

In [14]:
print(type(slct_tbl_full_df01))
print(slct_tbl_full_df01.shape)
display(slct_tbl_full_df01.head(11))
#display(slct_tbl_full_df01['article_text'][0])

<class 'pandas.core.frame.DataFrame'>
(28309, 8)


,text_id,source_name,author,title,url,publish_date,article_text,content
0,1,Breitbart News,"Lucas Nolan, Lucas Nolan",Nerd War!: Elon Musk and Mark Zuckerberg Agree...,https://www.breitbart.com/tech/2023/06/22/nerd...,2023-06-22T13:06:40Z,"\nElon Musk and Mark Zuckerberg, two of the wo...","Elon Musk and Mark Zuckerberg, two of the worl..."
1,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...
2,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...
3,4,Business Insider,Amanda Smith,"It costs over $300,000 to raise a child. For l...",https://www.businessinsider.com/personal-finan...,2023-06-22T21:05:44Z,Our experts answer readers' investing question...,Our experts answer readers' investing question...
4,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...
5,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...
6,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...
7,8,USA Today,Lebron Wire,Cowherd: Lakers are delusional for wanting to ...,https://lebronwire.usatoday.com/2023/06/29/cow...,2023-06-29T18:00:40Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nCowherd: Lakers ar...,One of the bright spots over the last two seas...
8,9,Breitbart News,"Sean Moran, Sean Moran","Democrat Bob Casey Meets with Soros Connected,...",https://www.breitbart.com/politics/2023/06/22/...,2023-06-22T21:18:35Z,\nSen. Bob Casey’s (D-PA) office met with a gr...,Sen. Bob Casey’s (D-PA) office met with a grou...
9,10,CNN,Parija Kavilanz,"Walgreens profit tumbles, slashes guidance ami...",https://www.cnn.com/2023/06/27/business/walgre...,2023-06-27T15:05:02Z,Walgreens Boots Alliance on Tuesday reported e...,Walgreens Boots Alliance on Tuesday reported e...


In [15]:
source_name_lst = slct_tbl_full_df01['source_name'].unique()
print(source_name_lst)

for s in source_name_lst:
    df01 = slct_tbl_full_df01.loc[slct_tbl_full_df01['source_name'] == s]
    df01 = df01.reset_index()
    counter = 0
    for c in range(0,2):
        counter += 1
        print(f'{s} article #{counter}')
        try:
            print(df01['url'][c])
            print(df01['author'][c])
            print(df01['article_text'][c], '\n')
        except:
            print(f'Skip {c}')

['Breitbart News' 'USA Today' 'Business Insider' 'CNN' 'MSNBC' 'ABC News'
 'Buzzfeed' 'NBC News' 'Wired' 'The Washington Post' 'Fox News' 'Reuters'
 'New York Post' 'Forbes' 'PEOPLE' 'CNBC' 'Vox' 'msnNOW']
Breitbart News article #1
https://www.breitbart.com/tech/2023/06/22/nerd-war-elon-musk-and-mark-zuckerberg-agree-to-cage-fight/
Lucas Nolan, Lucas Nolan

Elon Musk and Mark Zuckerberg, two of the world’s leading tech billionaires, recently engaged in a humorous social media exchange about a potential cage fight. Zuckerberg has been training in mixed martial arts for some time now, it is unclear if Musk has any combat experience whatsoever. When Musk proposed a match, Zuck responded, “Send me location.”
BBC News reports that Elon Musk, the owner of Twitter and CEO of Tesla and SpaceX, and Mark Zuckerberg, the CEO of Facebook (now known as Meta), have seemingly agreed to compete in a cage fight after engaging in a surprising Twitter conversation. The back and forth, which has since gon

### Commit changes and close cursor and connection instances

This step is integral to committing any remaining MySQL calls, as well as releasing the connection.

In [16]:
# Commit the changes to the database
db_conn.commit()

# Close the cursor and database connection
cursor.close()
db_conn.close()